# DCGAN with CelebA

## 1. Data

```script
chmod +x download.sh
./download.sh
unzip -q CelebA_128crop_FD.zip?dl=0 -d ./data/
```

## 2. Import Libs

In [ ]:
from __future__ import print_function
import torch
import os
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as tutils
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
random.seed(SEED)

## 3. Hyperparameters

In [ ]:
IMAGE_PATH = '../input/celeba-dataset/img_align_celeba/'
SAMPLE_PATH = '../'

if not os.path.exists(SAMPLE_PATH):
    os.makedirs(SAMPLE_PATH)

In [11]:
# Number of Workers for dataloader
workers = 2

# Batch size for training
batch_size = 128

# Image size
image_size = 64

# Number of channels of image 
channels = 3

# Number of latent vectors i.e output of generator
latent_vectors = 100

# Size of Feature map in generator
ngf = 64

# Size of Feature map in Discriminator
ndf = 64

# Number of Epochs
num_epochs = 5

# Learning Rate 
lr = 0.0002

# Beta1 HyperParamter for Adam Optimizer 
beta1 = 0.5



## 4. Load Data

In [ ]:
# Create the Dataset
dataset = datasets.ImageFolder(IMAGE_PATH, 
                             transform=transforms.Compose([
                                 transforms.Resize(image_size),
                                 transforms.CenterCrop(image_size),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                             ]))

# Create the dataloader
dataloader = tutils.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 5. Visualize Data

In [ ]:
real_data = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.title('Training Images')
plt.imshow(np.transpose(vutils.make_grid(real_data[0].to(device)[:64], padding=2, normalize=True).cpu(), (1, 2, 0)))
plt.show()

## 6. Initializing Weights

In [ ]:
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
        

## 7. Defining Architecture

### a) Generator

In [12]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_vectors, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),
            # Size will be ( ngf*8 x 4 x 4 )
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            # Size will be ( ngf*4 x 8 x 8 )
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            # Size will be ( ngf*2 x 16 x 16 )
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # size will be ( ngf x 32 x 32 )
            nn.ConvTranspose2d(ngf, channels, 4, 2, 1, bias=False ),
            nn.Tanh()    
        )
    
    def forward(self, x):
        return self.main(x)

In [13]:
generator_model = Generator().to(device)

generator_model.apply(init_weights)

print(generator_model)

TypeError: __init__() got an unexpected keyword argument 'bais'